Information about LZKN archives in Contra Hard Corps ROMs
---
Script open all LZKN archives in Contra Hard Corps ROM and calculate their sizes.<br>
It uses lzkn1.dll lib https://github.com/spiiin/lzkn to calculate sizes or archives and file lzkn1_list.txt to read addresses of all array, and lzkn1_dll_x86.dll (written by r57shell) to calculate expected repacked size of arrays.<br>
(Original addresses was collected with lua-script, that dumps addressess while decompressing data in emulator)<br>

In [1]:
from ctypes import *
from IPython.display import display
from ipy_table import *

ROM_NAME = "Contra - Hard Corps (U) [!].gen"
TXT_NAME = "lzkn1_list.txt"

CHUNK_SIZE = 8192
MAX_OUT_SIZE = 10**6

lzkn1_lab313 = cdll.LoadLibrary("lzkn1.dll")
lzkn1_r57shell = cdll.LoadLibrary("lzkn1_dll_x86.dll")

with open(ROM_NAME, "rb") as f:
    data = f.read()
data = map(ord, data)

with open(TXT_NAME, "rt") as f:
    lines = f.readlines()
lines = list(x.strip() for x in lines)

toTable =[]
toTable.append([u"LZKN address", u"Decompressed size", u"Original ROM compressor", u"LZKN compressor [lab313]", u"LZKN compressor [r57Shell]", u"delta(r57shell - lab313)"])
    
for line in lines:
    curAddres = int(line, 16)
    chunkData = data[curAddres:curAddres+CHUNK_SIZE]
    compressedSize = c_int()
    c_data = (c_ubyte * len(chunkData)) (*chunkData)
    c_data_p = POINTER(c_ubyte) (c_data)
    c_uncompressedData = create_string_buffer(MAX_OUT_SIZE)
    decompressedSize = lzkn1_lab313.decompress(c_data_p, byref(c_uncompressedData), byref(compressedSize))
    
    #  with open("map_" + hex(curAddres), "wb") as f:
    #      f.write(c_uncompressedData[:decompressedSize])
    
    c_recompressedData = create_string_buffer(decompressedSize)
    recompressedSize = lzkn1_lab313.compress(byref(c_uncompressedData), byref(c_recompressedData), decompressedSize)
    
    #  with open("lzkn_" + hex(curAddres), "wb") as f:
    #      f.write(c_recompressedData[:recompressedSize])
    
    c_recompressedData2 = create_string_buffer(decompressedSize)
    recompressedSize2 = lzkn1_r57shell.compress(byref(c_uncompressedData), byref(c_recompressedData2), decompressedSize)
    
    #print "Lzkn address: %8s"%hex(curAddres),
    #print " Decompressed size: %4d"%decompressedSize,
    #print " Compressed size: %4d"%compressedSize.value,
    #print " Recompressed size: %4d"%recompressedSize,
    #print " Difference with original: %4d"%(compressedSize.value - recompressedSize)

    toTable.append(
        [hex(curAddres),
         decompressedSize,
         compressedSize.value, 
         u"%d (%d)"%(recompressedSize, (recompressedSize - compressedSize.value )),
         u"%d (%d)"%(recompressedSize2, (recompressedSize2 - compressedSize.value )),
         u"%d" % (recompressedSize2 - recompressedSize)
         ]
    )
    
t = make_table(toTable)
t.apply_theme("basic")
display(t)
    



LZKN address,Decompressed size,Original ROM compressor,LZKN compressor [lab313],LZKN compressor [r57Shell],delta(r57shell - lab313)
0x9ed6,1536,893,871 (-22),854 (-39),-17
0x12a000,8192,4515,4394 (-121),4263 (-252),-131
0x12b1a4,8192,3293,3199 (-94),3092 (-201),-107
0x12c5a0,8192,2384,2324 (-60),2254 (-130),-70
0x12cef0,4096,1915,1862 (-53),1795 (-120),-67
0x12d66c,1024,348,346 (-2),337 (-11),-9
0x12d7c8,8192,4888,4752 (-136),4611 (-277),-141
0x12eae0,8192,5920,5782 (-138),5690 (-230),-92
0x130200,8192,4743,4637 (-106),4537 (-206),-100
0x131488,4096,2334,2276 (-58),2210 (-124),-66
